In [1]:
import networkx as nx
import glob
from collections import defaultdict, Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
fnames = glob.glob("/data/shruti/conspirecovery/reddit/initial/network/subreddit_linkshare00000*.gz")
lstframe = []
for f in fnames:
    temp_F = pd.read_csv(f, header=0)
    lstframe.append(temp_F)

In [3]:
frame = pd.concat(lstframe)
frame.head()

,url,f0_
0,https://fortheloveofbands.com/2019/01/09/how-t...,"WeAreTheMusicMakers,spotify"
1,https://i.redd.it/aia31x84zxe21.jpg,"crafts,StarWars"
2,https://www.rawstory.com/2018/08/watch-sacha-b...,"politics,milwaukee"
3,https://www.ndtv.com/allahabad-news/kumbh-shah...,"AutoNewspaper,NDTVauto"
4,https://themerkle.com/bitcoin-cash-price-tops-...,"CryptoCurrency,AllThingsCrypto"


In [4]:
list(frame)

['url', 'f0_']

In [5]:
climateskeptic = ["climateskeptics","greenskeptics","climatechangebullshit","climatehoax","Globalwarmingisfake","climateconspiratard"]
chemprints = ["CHEMPRINTS","Chemtrail","chemtrails","chemtrailpilots"]
flatearth = ["notaglobe","theworldisflat","flatearth", "Flat_Earth", "Flatearther"]
proclimate = ["ClimateCrisis", "CarbonCapture", "Climate_Discussion", "Climate", "ClimateActionPlan", "ClimateOffensive", "climatechange"]


In [6]:
def has_cd(sblst):
    for c in climateskeptic:
        if c in sblst:
            return 1
        
frame['is_cd'] = frame['f0_'].parallel_apply(lambda x: has_cd(x))

In [7]:
frame = frame.loc[frame['is_cd']==1]
print(len(frame))

6831


In [8]:
nodes = defaultdict(int)
edges = defaultdict(int)

for idx, row in frame.iterrows():
    sublist = row['f0_'].split(",")
    for s in sublist:
        nodes[s]+=1
        
    for i in range(len(sublist)-1):

        for j in range(i+1, len(sublist)):
            etuple = sorted([sublist[i], sublist[j]])
            edges[(etuple[0], etuple[1])]+=1
        

In [ ]:
df = pd.Series(edges).reset_index()   
df.columns = ['n1', 'n2', 'weight']   
df.head()

In [ ]:
len(df)

In [ ]:
len(df.loc[df['weight']>4])

In [ ]:
df = df.loc[df['weight']>4]

In [ ]:
cooc = pd.pivot_table(df, values='weight', index='n1', columns='n2').reset_index()
cooc.head()

In [ ]:
len(cooc)

In [ ]:
sublist = list(cooc)[1:]

In [ ]:
for s in sublist:
    cooc[s] = cooc[s].fillna(0)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import pickle
from collections import Counter, defaultdict
from itertools import combinations

import numpy as np
import scipy as sp
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import linalg
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import networkx as nx

G = nx.Graph()

# for n in nodes.keys():
#     G.add_node(n, size=nodes[n], value=domainvalues[n])

ndlist = []
    
for e in edges:
    if nodes[e[0]] > 5 and nodes[e[1]] > 5:
        if edges[e] > 2:
            if e[0] not in ndlist:
                try:
                    G.add_node(e[0], size=nodes[e[0]])
                except:
                    G.add_node(e[0], size=nodes[e[0]])
                ndlist.append(e[0])
            if e[1] not in ndlist:
                try:
                    G.add_node(e[1], size=nodes[e[1]])
                except:
                    G.add_node(e[1], size=nodes[e[1]])
                ndlist.append(e[1])


            G.add_edge(e[0], e[1], weight=edges[e])

In [ ]:
nx.write_gexf(G, "./network_new.gexf", encoding='utf-8')